## 설명
* 이마트몰 밀키트 카테고리(ID: 6000217707) 에서 판매처/상품명/가격 스크래핑

### Code Assist
* python code AI assistant : claude

### revision
* 1 : 이마트몰 밀키트 카테고리에서 판매처와 상품명, 가격 스크래핑
* 2 : 밀키트 카테고리 검색시 판매량 순으로 정렬 및 순위 지정. 한국식 음식으로 카테고리 분류
* 3
  * 평점, 리뷰수 수집 추가
  * 전처리 : '판매처'명이 "--" 인 경우 상품명에서 추출
  * 전처리 : 상품명에 무게를 뜻하는 'g'이 표기된 것이 많아 해당 무게값 제거

### 환경 : import

In [1]:
import requests
import logging
import time
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

### 환경 : 로깅 설정

In [2]:
# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.FileHandler('mealkit_emartmall_prdct_crawler.log'),
        logging.StreamHandler()
    ]
)

### 환경 : 전역 변수 선언

In [13]:
csv_filename = 'mealkit_products_salesranked_emart.csv'

### 함수 : 상품명 전처리
* 상품명 끝에 g 또는 G 문자가 있는 경우 무게값으로 간주하여 바로 앞 공백까지 삭제

In [5]:
# prompt: 파라미터로 전달되는 상품명에 대해 앞위 공백 제거 후 끝 문자가 g 또는 G 인 경우 바로 앞 공백까지 제거하여 반환하는 함수

def preprocess_product_name(product_name):
    """
    상품명에 대한 전처리 함수.

    Args:
        product_name: 전처리할 상품명.

    Returns:
        전처리된 상품명.
    """
    product_name = product_name.strip()  # 앞뒤 공백 제거
    if product_name and (product_name[-1] == 'g' or product_name[-1] == 'G'):
        # 끝 문자가 g 또는 G 이면서 상품명이 비어있지 않은 경우
        for i in range(len(product_name) - 2, -1, -1):
            if product_name[i] == ' ':
                product_name = product_name[:i]
                break
        else:
            product_name = product_name[:-1]  # 공백 없으면 g 또는 G만 제거
    return product_name

### 함수 : 상품명에 대한 한국 음식 카테고리명 반환

In [6]:
# prompt: 파라미터로 전달되는 상품명을 기준으로 한국식 음식으로 카테고리 분류 후 카테고리명 방환 함수

def categorize_korean_food(product_name):
    """
    상품명을 기준으로 한국식 음식 카테고리를 분류합니다.

    Args:
        product_name: 상품명.

    Returns:
        카테고리명 (str).
    """
    product_name = product_name.lower()  # 소문자 변환으로 대소문자 구분 없이 처리

    # 카테고리 분류 규칙 (예시) - 필요에 따라 추가 및 수정
    if any(keyword in product_name for keyword in ["쌀국수", "베트남식"]):
        return "베트남식"
    elif any(keyword in product_name for keyword in ["찌개", "김치찌개", "된장찌개", "부대찌개", "순두부찌개"]):
        return "찌개류"
    elif any(keyword in product_name for keyword in ["볶음밥", "비빔밥", "김밥", "잡채", "볶음"]):
        return "밥/면류"
    elif any(keyword in product_name for keyword in ["불고기", "갈비", "제육볶음", "돼지갈비", "닭갈비"]):
        return "고기류"
    elif any(keyword in product_name for keyword in ["전", "빈대떡", "파전", "김치전"]):
        return "전류"
    elif any(keyword in product_name for keyword in ["찜", "갈비찜", "닭볶음탕"]):
        return "찜류"
    elif any(keyword in product_name for keyword in ["국", "미역국", "된장국", "떡국"]):
        return "국류"
    elif any(keyword in product_name for keyword in ["구이", "삼겹살", "목살"]):
        return "구이류"
    elif any(keyword in product_name for keyword in ["닭", "닭볶음탕", "닭갈비"]):
        return "닭요리"
    elif any(keyword in product_name for keyword in ["해물", "생선", "회"]):
        return "해물/생선류"
    elif any(keyword in product_name for keyword in ["짬뽕", "짜장", "탕수육"]):
        return "중식"
    elif any(keyword in product_name for keyword in ["오꼬노미야끼", "야키소바", "라멘"]):
        return "일식"
    elif any(keyword in product_name for keyword in ["떡볶이", "순대", "튀김", "김말이"]):
        return "분식류"
    elif any(keyword in product_name for keyword in ["김치", "된장", "깍두기", "열무김치", "배추김치"]):
        return "기타한식류"
    else:
        return "기타"

### 함수 : 이마트 상품 단위 페이지 크롤링

In [9]:
# prompt: Request URL:
# https://emart.ssg.com/disp/ajaxCategory.ssg?dispCtgId=6000217707&sort=sale&page=
# Request Method:
# GET
# 에서 각 페이지의 <li class="mnemitem_grid_item"> 요소에서 아래 사항을 데이터프레이으로 반환하는데,
# '순서'라는 컬럼명으로 1번부터 시작하는 값 저장
# 상품ID 컬럼에 <div class="mnemitem_unit "> 아래 data-react-unit-id 값 저장
# 판매처 컬럼에 <div class="mnemitem_tit "> 아래 <span class="mnemitem_goods_brand"> 값을 저장하는데, 없는 경우 공백으로 저장
# 상품명 컬럼에 <div class="mnemitem_tit "> 아래 <span class="mnemitem_goods_tit"> 값 저장
# 가격 컬럼에 <div class="mnemitem_prd_per"> 아래 <em class="ssg_price"> 값을 정수로 저장
# 평점 컬럼에 <div class="mnemitem_review_score"> 아래 첫번째 <span class="review_text"> 값을 저장하는데, 없는 경우 공백으로 저장
# 리뷰수 컬럼에 <div class="mnemitem_review_score"> 아래 두번째 <span class="review_text"> 값을 저장하는데, 없는 경우 공백으로 저장
# 100g당 가격에 <div class="unit_price">100g 당 1,147원</div> 에서 1,147값을 저장하는데, 없는 경우 공백으로 저장
# 하는 파이썬 코드

def get_product_info(page_num):
    """
    지정된 페이지에서 상품 정보(순서, 판매처, 상품명, 가격)를 수집하는 함수

    Args:
        page_num (int): 수집할 페이지 번호

    Returns:
        list: 상품 정보 (순서, 상품ID, 판매처, 상품명, 가격, 평점, 리뷰수, 100g당 가격) 튜플의 리스트
    """
    url = f"https://emart.ssg.com/disp/ajaxCategory.ssg?dispCtgId=6000217707&sort=sale&page={page_num}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        products = []
        items = soup.select('li.mnemitem_grid_item')
        start_index = (page_num - 1) * len(items) + 1

        for idx, item in enumerate(items, start=start_index):
            try:
                product_nm_tmp = item.select_one('span.mnemitem_goods_tit').text.strip()

                product_id = item.select_one('div.mnemitem_unit')['data-react-unit-id']
                brand_nm = item.select_one('span.mnemitem_goods_brand')
                brand_nm = brand_nm.text.strip() if brand_nm else ""
                product_nm = preprocess_product_name(product_nm_tmp)

                if brand_nm == "" and product_nm.startswith('['):
                    end_bracket = product_nm.find(']')
                    if end_bracket != -1:
                        brand_nm = product_nm[1:end_bracket]
                        product_nm = product_nm[end_bracket + 1:].strip()

                price = int("".join(filter(str.isdigit, item.select_one('em.ssg_price').text.strip())))
                review_scores = item.select('span.review_text')
                rating = review_scores[0].text.strip() if len(review_scores) > 0 else ""
                review_count = review_scores[1].text.strip() if len(review_scores) > 1 else ""
                review_count = "".join(filter(str.isdigit, review_count))
                unit_price_elem = item.select_one('div.unit_price')
                unit_price = "".join(filter(str.isdigit, unit_price_elem.text.strip())) if unit_price_elem else ""
                categorize_nm = categorize_korean_food(product_nm)

                products.append([idx, product_id, brand_nm, product_nm, price, rating, review_count, unit_price, categorize_nm])

            except Exception as e:
                print(f"상품 정보 추출 중 오류 (순서: {idx}): {str(e)}")
                continue
        return products

    except requests.exceptions.RequestException as e:
        print(f"페이지 {page_num} 요청 중 오류 발생: {str(e)}")
        return []

### 함수 : 여러 페이지의 상품 정보 수집

In [10]:
def scrape_multiple_pages(start_page, end_page):
    """
    여러 페이지의 상품 정보를 수집하는 함수

    Args:
        start_page (int): 시작 페이지 번호
        end_page (int): 끝 페이지 번호

    Returns:
        pandas.DataFrame: 수집된 상품 정보
    """
    all_products = []
    for page in range(start_page, end_page + 1):
        print(f"페이지 {page} 수집 중...")
        products = get_product_info(page)
        all_products.extend(products)
        time.sleep(2)  # 서버 부하를 줄이기 위한 딜레이 (2초)

    df = pd.DataFrame(all_products, columns=['순서', '상품ID', '판매처', '상품명', '가격', '평점', '리뷰수', '100g당 가격', '분류'])
    df = df.drop_duplicates()
    df = df.sort_values('순서')
    return df

## 실행 : 크롤링

In [14]:
def main():
    """
    메인 실행 함수
    """
    try:
        start_time = datetime.now()
        print(f"수집 시작: {start_time}")

        # 1페이지부터 전체 12페이지까지 수집
        df = scrape_multiple_pages(1, 12)

        # 결과 저장
        output_file = csv_filename
        df.to_csv(output_file, index=False, encoding='utf-8-sig')

        # 통계 출력
        end_time = datetime.now()
        duration = end_time - start_time

        print(f"\n수집 완료!")
        print(f"시작 시간: {start_time}")
        print(f"종료 시간: {end_time}")
        print(f"소요 시간: {duration}")
        print(f"총 수집 상품 수: {len(df)}개")
        print(f"저장 파일명: {output_file}")

    except Exception as e:
        print(f"프로그램 실행 중 오류 발생: {str(e)}")

if __name__ == "__main__":
    main()

수집 시작: 2025-01-13 12:45:29.344622
페이지 1 수집 중...
페이지 2 수집 중...
페이지 3 수집 중...
페이지 4 수집 중...
페이지 5 수집 중...
페이지 6 수집 중...
페이지 7 수집 중...
페이지 8 수집 중...
페이지 9 수집 중...
페이지 10 수집 중...
페이지 11 수집 중...
페이지 12 수집 중...

수집 완료!
시작 시간: 2025-01-13 12:45:29.344622
종료 시간: 2025-01-13 12:46:18.630661
소요 시간: 0:00:49.286039
총 수집 상품 수: 943개
저장 파일명: mealkit_products_salesranked_emart.csv
